**Test to see if all network components are working as expected**

In [1]:
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config

In [8]:
class NebulaHandler:
    def __init__(self, space_name, host='nebula-docker-compose-graphd-1', port=9669, user='root', password='nebula'):
        """
        Initializes the NebulaHandler object and establishes a connection to NebulaGraph.

        Args:
            space_name (str): The name of the space to initialize and switch to.
            host (str): Host address of the NebulaGraph server.
            port (int): Port of the NebulaGraph server.
            user (str): Username for NebulaGraph.
            password (str): Password for NebulaGraph.
        """
        self.space_name = space_name
        self.config = Config()
        self.connection_pool = ConnectionPool()
        if not self.connection_pool.init([(host, port)], self.config):
            raise Exception("Failed to initialize the connection pool.")
        self.session = self.connection_pool.get_session(user, password)
        self.session.execute(f"USE {self.space_name}")

    def execute_query(self, query):
        """Executes a NebulaGraph query and returns the result."""
        try:
            result = self.session.execute(query)
            if result.error_code() != 0:
                raise Exception(f"Error executing query: {result.error_msg()}")
            return result
        except Exception as e:
            print(f"Exception occurred: {e}")
            return None

    def show_spaces(self):
        """Fetches all spaces and returns their names."""
        query = """
            SHOW SPACES;
        """
        result = self.execute_query(query)
        return result

In [9]:
handler = NebulaHandler(space_name='test_space', host='nebula-docker-compose-graphd-1', port=9669, user='root', password='nebula')